In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *

import time

Setting environment for Gordon


In [ ]:
# Compute Gradient - score volume

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD589']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

#     (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
#     np.loadtxt(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}), dtype=np.int)
            
    volume2_allLabels = {}

    for name in labels_unsided:

        if name == 'BackG':
            continue

        volume2_roi = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_%(label)s.bp' % \
                                                          {'stack': stack, 'label': name})).astype(np.float16)
        volume2_allLabels[name] = volume2_roi
        del volume2_roi

#     test_ydim, test_xdim, test_zdim = volume2_allLabels.values()[0].shape
#     test_centroid = np.array([.5*test_xdim, .5*test_ydim, .5*test_zdim])

    t1 = time.time()

    for name in labels_unsided:

        if name == 'BackG':
            continue

        t = time.time()

        gy_gx_gz = np.gradient(volume2_allLabels[name].astype(np.float32), 3, 3, 3) 
        # 3.3 second, much faster than loading

        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

#         bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
#         bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

In [8]:
# Compute Gradient - Annotation Volume

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD594']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

    volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume.bp' % \
                                                 {'stack': stack}))
    
    label_indices = set(np.unique(volume)) - {0}
            
#     volumes_allLabels = {labels_sided[i-1]: np.zeros_like(volume) for i in label_indices}

    structure_bboxes = {}
    with open(os.path.join(volume_dir, stack, stack+'_annotationVolume_landmarkBboxes.txt'), 'r') as f:
        for ln in f.readlines():
            ln_split = ln.split()
            structure_bboxes[ln_split[0]] = map(int, ln_split[1:]) # xmin,xmax,ymin,ymax,zmin,zmax

    t1 = time.time()
    
    for i in label_indices:
        
        name = labels_sided[i-1]
        
        xmin, xmax, ymin, ymax, zmin, zmax = structure_bboxes[name]
        
        mask_inBbox = volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == i
                
        volume_oneName = np.zeros((ymax+1-ymin, xmax+1-xmin, zmax+1-zmin), np.float16)
        volume_oneName[mask_inBbox] = i

        t = time.time()
        
        try:
            gy_gx_gz = np.gradient(volume_oneName, 3, 3, 3) 
        # 3.3 second, much faster than loading
        except:
            raise Exception('Error in computing gradient for %s.\n' % name)

        gx_padded = np.zeros_like(volume)
        gx_padded[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz[1]
        gy_padded = np.zeros_like(volume)
        gy_padded[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz[0]
        gz_padded = np.zeros_like(volume)
        gz_padded[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] = gy_gx_gz[2]
        
        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_padded, os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name}))
        bp.pack_ndarray_file(gx_padded, os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name}))
        bp.pack_ndarray_file(gz_padded, os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name}))

        del gy_gx_gz, gx_padded, gy_padded, gz_padded, volume_oneName

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

gradient 12N: 2.666507 seconds
save 12N: 0.137535 seconds
gradient 5N_L: 2.674212 seconds
save 5N_L: 0.135570 seconds
gradient 5N_R: 2.433297 seconds
save 5N_R: 0.145240 seconds
gradient 6N_L: 6.053854 seconds
save 6N_L: 0.141592 seconds
gradient 6N_R: 2.033209 seconds
save 6N_R: 0.139705 seconds
gradient 7N_L: 4.907842 seconds
save 7N_L: 0.142288 seconds
gradient 7N_R: 2.047576 seconds
save 7N_R: 0.143488 seconds
gradient 7n_L: 5.224035 seconds
save 7n_L: 0.144438 seconds
gradient 7n_R: 3.282860 seconds
save 7n_R: 0.142224 seconds
gradient AP: 2.537012 seconds
save AP: 0.160608 seconds
gradient Amb_L: 2.444107 seconds
save Amb_L: 0.145693 seconds
gradient Amb_R: 4.246033 seconds
save Amb_R: 0.145752 seconds
gradient LC_L: 2.990531 seconds
save LC_L: 0.135605 seconds
gradient LC_R: 2.313436 seconds
save LC_R: 0.139901 seconds
gradient LRt_L: 3.020620 seconds
save LRt_L: 0.151490 seconds
gradient LRt_R: 5.664793 seconds
save LRt_R: 0.156634 seconds
gradient Pn_L: 3.555673 seconds
save P

MD594


ValueError: Shape of array too small to calculate a numerical gradient, at least two elements are required.

In [11]:
volume_oneName.shape

(68, 51, 1)

In [ ]:
# Compute Gradient - Annotation Volume

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD589']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

    volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_annotationVolume.bp' % \
                                                 {'stack': stack}))
    
    label_indices = set(np.unique(volume)) - {0}
            
    volumes_allLabels = {labels_sided[i-1]: np.zeros_like(volume) for i in label_indices}
    for i in label_indices:
        mask = volume == i
        volumes_allLabels[labels_sided[i-1]][mask] = volume[mask]
        del mask

    t1 = time.time()

    for name in volumes_allLabels.iterkeys():

        t = time.time()

        gy_gx_gz = np.gradient(volumes_allLabels[name].astype(np.float32), 3, 3, 3) 
        # 3.3 second, much faster than loading

        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_annotationVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))